# Dynamic Quantum Chemistry Workflow 

In this tutorial, we discuss how _covalent_ can be used to construct and manage dynamic workflows that are common in quantum-chemical simulations. In this example, given a crystal structure, we seek to do the following:

1. Relax the atomic positions and cell volume for the structure (i.e. find the local minimum energy configuration).
2. Carve all possible surface slabs of the relaxed structure.
3. For each generated slab, relax the atomic positions of each structure.

This will be done using the materials informatics library [Pymatgen](https://github.com/materialsproject/pymatgen) and a machine learned interatomic potential called [M3GNet](https://www.nature.com/articles/s43588-022-00349-3). Because the number of slabs is only known at runtime, this is a [dynamic workflow](https://covalent.readthedocs.io/en/latest/developer/patterns/dynamic_workflow.html) and we must take advantage of the [sublattice](https://covalent.readthedocs.io/en/latest/glossary/index.html#term-Sublattice) feature in Covalent to properly dispatch the workflow.

First, we need to install the necessary Python packages from the provided `requirements.txt` file (e.g. `pip install -r requirements.txt`). These are printed below:

In [1]:
with open("requirements.txt", "r") as file:
    for line in file:
        print(line.rstrip())

covalent
pymatgen[relaxation]==2023.5.10


Once that is done, we can import the necessary functions for this tutorial:

In [2]:
import covalent as ct

from subprocess import Popen
from pymatgen.core import Structure
from pymatgen.core.surface import generate_all_slabs

We then will make sure the Covalent server is started:

In [3]:
sts = Popen("covalent start", shell=True).wait()

Covalent server has started at http://localhost:48008


Now we will define the individual compute tasks. The first `Electron` we will define is a `relax_structure` function, which takes in a Pymatgen `Structure` object as the input and runs the relaxation calculation using M3GNet. It returns an updated `Structure` object. This process can be thought of as the compute-heavy task.

In [4]:
@ct.electron
def relax_structure(structure, relax_cell=True):
    return structure.relax(relax_cell=relax_cell)

Now we will define a separate `Electron` called `carve_slabs` that takes in a `Structure` object and carves all plausible surface slabs, which it returns as a list of new `Structure` objects. This is a relatively quick task, but we still need to define it as an `Electron` so that it can be used appropriately in the workflow.

In [5]:
@ct.electron
def carve_slabs(structure, max_index=1, min_slab_size=10.0, min_vacuum_size=10.0):
    slabs = generate_all_slabs(
        structure,
        max_index,
        min_slab_size,
        min_vacuum_size,
    )
    return slabs

Now, for the most subtle but important bit. We need to define a sublattice that will take in the list of slabs and relax each individual slab using the previously defined `relax_structure` `Electron`. The sublattice is crucial here because the number of slabs is only determined at runtime and will vary depending on the input `Structure` object.

In [6]:
@ct.electron
@ct.lattice
def relax_slabs(slabs):
    return [relax_structure(slab, relax_cell=False) for slab in slabs]

With these individual `Electron` objects defined, we can now define the overall workflow that stitches them together and forms a `Lattice`. As introduced at the start of this tutorial, there are three major sets of tasks, and those are reflected by the three functions defined above.

In [7]:
@ct.lattice(executor="local")
def workflow(structure):
    relaxed_structure = relax_structure(structure)
    slabs = carve_slabs(relaxed_structure)
    relaxed_slabs = relax_slabs(slabs)
    return relaxed_slabs

Now we can create an example `Structure` as the input for our workflow. Its structure is shown below.

In [8]:
structure = Structure(
    lattice=[[0, 2.13, 2.13], [2.13, 0, 2.13], [2.13, 2.13, 0]],
    species=["Mg", "O"],
    coords=[[0, 0, 0], [0.5, 0.5, 0.5]],
)

And we finally take this input `Structure` and run it through our newly created workflow.

In [9]:
dispatch_id = ct.dispatch(workflow)(structure)
results = ct.get_result(dispatch_id, wait=True)


Lattice Result
status: COMPLETED
result: [Structure Summary
Lattice
    abc : 3.012274887854692 3.0122748878546917 30.122748878546922
 angles : 120.00000000000001 120.00000000000001 60.00000000000001
 volume : 193.27193999999986
      A : 2.6087065760640837 0.0 -1.5061374439273467
      B : 0.8695688586880268 2.459512146747805 -1.5061374439273467
      C : 0.0 0.0 30.122748878546922
    pbc : True True True
PeriodicSite: Mg (-0.0808, -0.0571, 2.1193) [-0.0232, -0.0232, 0.0680]
PeriodicSite: O (1.8010, 1.2735, -0.6459) [0.5178, 0.5178, 0.0303]
PeriodicSite: Mg (-0.0401, -0.0284, 5.2020) [-0.0115, -0.0115, 0.1715]
PeriodicSite: O (1.7790, 1.2579, 2.3282) [0.5114, 0.5114, 0.1284]
PeriodicSite: Mg (-0.0392, -0.0277, 8.2159) [-0.0113, -0.0113, 0.2716]
PeriodicSite: O (1.7778, 1.2571, 5.3384) [0.5111, 0.5111, 0.2283]
PeriodicSite: Mg (-0.0320, -0.0226, 11.2406) [-0.0092, -0.0092, 0.3722]
PeriodicSite: O (1.7829, 1.2607, 8.3596) [0.5126, 0.5126, 0.3288]
PeriodicSite: Mg (-0.0059, -0.0042, 14

The workflow in the Covalent UI looks like the following. Note that when you click on the sublattice in the UI, it will open up a new view that shows the individual tasks that were run (shown on the right).

![ui](assets/ui_animation.gif)

The image below shows the input structure on the left and one of the four generated slabs on the right.

![slabs](assets/slabgen.png)

And this is the output of the workflow:

In [ ]:
print(results)